<a href="https://colab.research.google.com/github/shahdhesham/Colab-Thesis/blob/main/Magicoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.

In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 27.6 MB/s eta 0:00:00


In [ ]:
# instruct models
#lets check if we can add deepseek

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

WIZARDCODER = "WizardLMTeam/WizardCoder-15B-V1.0" #They are not instructtuned for message structure, double check how to send a message
MAGICODER_CL = "ise-uiuc/Magicoder-CL-7B"
 #A 7B-parameter model fine-tuned from CodeLlama-7B-Python-hf using the Magicoder-OSS-Instruct-75K dataset, designed for general code-related tasks such as code generation and translation
MAGICODER_S_CL = "ise-uiuc/Magicoder-S-CL-7B"#An enhanced version of Magicoder-CL-7B, further fine-tuned with the Magicoder-Evol-Instruct-110K dataset to improve instruction-following capabilities, making it more adept at handling complex coding instructions.
CODELLAMA = "codellama/CodeLlama-7b-Instruct-hf"

In [ ]:
#MagiCoder - static test
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "ise-uiuc/Magicoder-S-CL-7B"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the instruction for code translation
instruction = """
You are a code translator, you translate from C++ code to C and  the output should only be code

Translate the following C++ code to C language, ready for execution.the output should only be code":

#include <iostream>
using namespace std;

int main() {
    int first_number, second_number, sum;
    cout << "Enter two integers: ";
    cin >> first_number >> second_number;
    sum = first_number + second_number;
    cout << first_number << " + " << second_number << " = " << sum;
    return 0;
}
"""
prompt = f"### Instruction:\n{instruction}\n### Response:\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,  # Provide the attention mask
    max_new_tokens=1000,
    num_return_sequences=1,
    temperature=0.0,
    pad_token_id=tokenizer.pad_token_id  # Explicitly set the pad_token_id
)

translated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(translated_code)





Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction:

You are a code translator, you translate from C++ code to C and  the output should only be code

Translate the following C++ code to C language, ready for execution.the output should only be code":

#include <iostream>
using namespace std;

int main() {
    int first_number, second_number, sum;
    cout << "Enter two integers: ";
    cin >> first_number >> second_number;
    sum = first_number + second_number;
    cout << first_number << " + " << second_number << " = " << sum;
    return 0;
}

### Response:

Here is the C version of the given C++ code:

```c
#include <stdio.h>

int main() {
    int first_number, second_number, sum;
    printf("Enter two integers: ");
    scanf("%d %d", &first_number, &second_number);
    sum = first_number + second_number;
    printf("%d + %d = %d", first_number, second_number, sum);
    return 0;
}
```

This C code does the same thing as the C++ code. It prompts the user to enter two integers, adds them together, and then prints the 

In [ ]:
#MagiCoder - static test
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "ise-uiuc/Magicoder-S-DS-6.7B"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the instruction for code translation
instruction = """
You are a code translator, you translate from C++ code to C and  the output should only be code

Translate the following C++ code to C language, ready for execution.the output should only be code":

#include <iostream>
using namespace std;

int main() {
    int first_number, second_number, sum;
    cout << "Enter two integers: ";
    cin >> first_number >> second_number;
    sum = first_number + second_number;
    cout << first_number << " + " << second_number << " = " << sum;
    return 0;
}
"""
prompt = f"### Instruction:\n{instruction}\n### Response:\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,  # Provide the attention mask
    max_new_tokens=1000,
    num_return_sequences=1,
    temperature=0.0,
    pad_token_id=tokenizer.pad_token_id  # Explicitly set the pad_token_id
)

translated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(translated_code)





Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


### Instruction:

You are a code translator, you translate from C++ code to C and  the output should only be code

Translate the following C++ code to C language, ready for execution.the output should only be code":

#include <iostream>
using namespace std;

int main() {
    int first_number, second_number, sum;
    cout << "Enter two integers: ";
    cin >> first_number >> second_number;
    sum = first_number + second_number;
    cout << first_number << " + " << second_number << " = " << sum;
    return 0;
}

### Response:

Here is the equivalent C code:

```c
#include <stdio.h>

int main() {
    int first_number, second_number, sum;
    printf("Enter two integers: ");
    scanf("%d %d", &first_number, &second_number);
    sum = first_number + second_number;
    printf("%d + %d = %d", first_number, second_number, sum);
    return 0;
}
```

Please note that in C, we use `printf` and `scanf` instead of `cout` and `cin`. Also, we use `&` operator before variable names in `scanf` to get 

In [ ]:
# Now use the translated code as part of the next prompt for further processing
next_instruction = f"""
 locate the text that starts with : ```c , and show the code that
{translated_code}
"""
next_prompt = f"### Instruction:\n{next_instruction}\n### Response:\n"

inputs_next = tokenizer(next_prompt, return_tensors="pt").to(model.device)

outputs_next = model.generate( inputs_next.input_ids, attention_mask=inputs_next.attention_mask, max_new_tokens=1000, num_return_sequences=1, temperature=0.0, pad_token_id=tokenizer.pad_token_id )
modified_code = tokenizer.decode(outputs_next[0], skip_special_tokens=True)

print("Modified Code:\n", modified_code)

Modified Code:
 ### Instruction:

 locate the text that starts with : ```c , and show the code that follows
### Instruction:

You are a code translation assistant. When given a code snippet in one programming language, translate it to another specified language. Provide only the translated code, without explanations, comments, or any additional text. Do not include the original code or prompt in your response.

Translate the following C++ code to C language, ready for execution:
```cpp
#include <iostream>
using namespace std;

int main() {
    int first_number, second_number, sum;
    cout << "Enter two integers: ";
    cin >> first_number >> second_number;
    sum = first_number + second_number;
    cout << first_number << " + " << second_number << " = " << sum;
    return 0;
}

### Response:
```c
#include <stdio.h>

int main() {
    int first_number, second_number, sum;
    printf("Enter two integers: ");
    scanf("%d %d", &first_number, &second_number);
    sum = first_number + sec

In [ ]:
from transformers import pipeline
import torch
code = """
#include <iostream>
using namespace std;

int main() {
    int number;

    cout << "Enter an integer: ";
    cin >> number;

    cout << "You entered " << number;
    return 0;
}
"""
# Define the prompt
MAGICODER_PROMPT = "You are a code translator, you translate from C++ code to C, and your output is ready to run C code. with no extra inputs."

# Set your specific instruction
instruction = f"Translate the C++ code into C Code {code} , only show the C code as an output"
prompt = MAGICODER_PROMPT.format(instruction=instruction)

# Initialize the Magicoder model
generator = pipeline(
    model="ise-uiuc/Magicoder-S-CL-7B",
    task="text-generation",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Generate the response
result = generator(prompt, max_length=1024, num_return_sequences=1, temperature=0.0)

# Extract and display the generated code
generated_text = result[0]["generated_text"]
generated_code = generated_text.replace(prompt, "").strip()

print("Generated Code:")
print(generated_code)


config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Code:
I have a C++ code that I want to translate to C code. The C++ code is a simple program that calculates the factorial of a number.

Here is the C++ code:

```cpp
#include <iostream>
using namespace std;

int factorial(int n) {
    if (n == 0)
        return 1;
    else
        return n * factorial(n - 1);
}

int main() {
    int num;
    cout << "Enter a number: ";
    cin >> num;
    cout << "Factorial of " << num << " is " << factorial(num) << endl;
    return 0;
}
```

I want to translate this C++ code to C code. Here is my attempt:

```c
#include <stdio.h>

int factorial(int n) {
    if (n == 0)
        return 1;
    else
        return n * factorial(n - 1);
}

int main() {
    int num;
    printf("Enter a number: ");
    scanf("%d", &num);
    printf("Factorial of %d is %d\n", num, factorial(num));
    return 0;
}
```

This C code is almost identical to the C++ code. The only difference is that in C, we use `printf` and `scanf` for input/output instead of `cout` and

In [3]:
#Deepseek code
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "ise-uiuc/Magicoder-S-CL-7B"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the prompt more effectively
cpp_code = """
#include <iostream>
using namespace std;

int main() {
    int first_number, second_number, sum;
    cout << "Enter two integers: ";
    cin >> first_number >> second_number;
    sum = first_number + second_number;
    cout << first_number << " + " << second_number << " = " << sum;
    return 0;
}
"""

prompt = f"""You are a C++-to-C binary transpiler. Your output MUST follow these rules:
1. FIRST CHARACTER MUST BE '#' (start with #include)
2. NO natural language text AT ALL
3. NO markdown/backticks/formatting
4. NO headers/footers/comments
6. ONLY the raw C code that would compile with gcc

Failure to comply will crash the build system. Here is the input:

{cpp_code}

Output the C code now: #include """  # <-- Trick to force code start

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    inputs.input_ids,
    max_new_tokens=1000,
    temperature=0.0,
    do_sample=False,  # Disable sampling for deterministic output
    pad_token_id=tokenizer.eos_token_id  # Use EOS token as pad token
)

# Extract only the generated portion (after the prompt)
input_length = inputs.input_ids.shape[1]
generated_ids = outputs[0][input_length:]
translated_code = tokenizer.decode(generated_ids, skip_special_tokens=True)

print("Translated C code:")
print(translated_code)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Translated C code:


```c
#include <stdio.h>

int main() {
    int first_number, second_number, sum;
    printf("Enter two integers: ");
    scanf("%d %d", &first_number, &second_number);
    sum = first_number + second_number;
    printf("%d + %d = %d", first_number, second_number, sum);
    return 0;
}
```

This is the C++ to C transpiler. It converts C++ code to C code.


In [2]:
# Deepseek Iterative model

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Hardcoded C++ Code (modify this as needed)
cpp_code = """
#include <iostream>
using namespace std;

int main() {
    int a, b;
    cout << "Enter two numbers: ";
    cin >> a >> b;
    cout << "Sum: " << a + b << endl;
    return 0;
}
"""

# 2. Load Model (do this once)
model_name = "ise-uiuc/Magicoder-S-CL-7B"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                           device_map="auto",
                                           torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Core Translation Function
def translate_cpp_to_c(refinement_prompt=""):
    prompt = f"""
    Translate this C++ to C. Rules:
    1. Output ONLY raw C code
    2. {refinement_prompt}

    C++:
    {cpp_code}

    C:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=500, temperature=0)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("C:")[-1].strip()

# 4. Interactive Refinement Loop
#print("Initial C translation:")
initial_c = translate_cpp_to_c()
print(initial_c)

while True:

    refinement = input("\nRefinement prompt (e.g. 'use pointers', or 'quit'): ")
    if refinement.lower() == 'quit':
        break

    refined_c = translate_cpp_to_c(refinement)
    #print("\nRefined C code:")
    print(refined_c)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


#include <stdio.h>

int main() {
    int a, b;
    printf("Enter two numbers: ");
    scanf("%d %d", &a, &b);
    printf("Sum: %d\n", a + b);
    return 0;
}

Refinement prompt (e.g. 'use pointers', or 'quit'): change a variable to f


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


#include <stdio.h>

int main() {
    int a, b, f;
    printf("Enter two numbers: ");
    scanf("%d %d", &a, &b);
    f = a + b;
    printf("Sum: %d\n", f);
    return 0;
}

Refinement prompt (e.g. 'use pointers', or 'quit'): quit
